In [2]:
using Pkg

In [3]:
cd("D:\\Home\\Git\\Oceananigans.jl\\")
pwd()
Pkg.activate(".")

"D:\\Home\\Git\\Oceananigans.jl\\Project.toml"

In [26]:
using Oceananigans, Oceananigans.Operators, BenchmarkTools

In [18]:
abstract type FieldSet end

struct VelocityFields <: FieldSet
    u::FaceFieldX
    v::FaceFieldY
    w::FaceFieldZ
end

function VelocityFields(g, T=Float64)
    u = FaceFieldX(g, T)
    v = FaceFieldY(g, T)
    w = FaceFieldZ(g, T)
    VelocityFields(u, v, w)
end

struct TemporaryFields <: FieldSet
    fC1::CellField
    fC2::CellField
    fC3::CellField
    ffX::FaceFieldX
    ffY::FaceFieldY
    ffZ::FaceFieldZ
end

function TemporaryFields(g, T=Float64)
    fC1 = CellField(g, T)
    fC2 = CellField(g, T)
    fC3 = CellField(g, T)
    fFX = FaceFieldX(g, T)
    fFY = FaceFieldY(g, T)
    fFZ = FaceFieldZ(g, T)
    TemporaryFields(fC1, fC2, fC3, fFX, fFY, fFZ)
end

TemporaryFields

In [40]:
function div2!(g::RegularCartesianGrid, fx::FaceFieldX, fy::FaceFieldY, fz::FaceFieldZ, div::CellField, tmp::TemporaryFields)
    δx!(g, fx, tmp.fC1)  # tmp.fC1 now stores δx(fx)
    δy!(g, fy, tmp.fC2)  # tmp.fC2 now stores δy(fy)
    δz!(g, fz, tmp.fC3)  # tmp.fC3 now stores δz(fz)
    @. div.data = (1/g.V) * ( g.Ax * tmp.fC1.data + g.Ay * tmp.fC2.data + g.Az * tmp.fC3.data )
    nothing
end

div2! (generic function with 1 method)

In [41]:
g = RegularCartesianGrid((100, 100, 100), (1000, 1000, 1000), Float64);
U = VelocityFields(g);
tmp = TemporaryFields(g);
divv = CellField(g);

In [42]:
@benchmark div2!(g, U.u, U.v, U.w, divv, tmp)

BenchmarkTools.Trial: 
  memory estimate:  78.63 KiB
  allocs estimate:  20
  --------------
  minimum time:     4.337 ms (0.00% GC)
  median time:      5.224 ms (0.00% GC)
  mean time:        5.419 ms (0.26% GC)
  maximum time:     11.681 ms (0.00% GC)
  --------------
  samples:          921
  evals/sample:     1

In [33]:
@benchmark δx!(g, U.u, tmp.fC1)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     1.092 ms (0.00% GC)
  median time:      1.198 ms (0.00% GC)
  mean time:        1.251 ms (0.00% GC)
  maximum time:     2.605 ms (0.00% GC)
  --------------
  samples:          3975
  evals/sample:     1

In [43]:
function div3!(g::RegularCartesianGrid, fx::FaceFieldX, fy::FaceFieldY, fz::FaceFieldZ, div::CellField, tmp::TemporaryFields)
    δxfx = CellField(g)
    δyfy = CellField(g)
    δzfz = CellField(g)
    δx!(g, fx, δxfx)
    δy!(g, fy, δyfy)
    δz!(g, fz, δzfz)
    @. div.data = (1/g.V) * ( g.Ax * δxfx.data + g.Ay * δyfy.data + g.Az * δzfz.data )
    nothing
end

div3! (generic function with 1 method)

In [44]:
@benchmark div3!(g, U.u, U.v, U.w, divv, tmp)

BenchmarkTools.Trial: 
  memory estimate:  22.97 MiB
  allocs estimate:  35
  --------------
  minimum time:     15.457 ms (3.53% GC)
  median time:      16.652 ms (22.98% GC)
  mean time:        16.909 ms (23.03% GC)
  maximum time:     22.654 ms (17.79% GC)
  --------------
  samples:          296
  evals/sample:     1